In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from vincenty import vincenty
from copy import deepcopy
from glob import glob
from pprint import pprint

In [2]:
colnames = ['lat','long', 'null', 'alt' ,'DateTime','Date','Time']
df = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/unsupervised-learning-metis/Geolife Trajectories 1.3/Data/013/Trajectory/20080929001436.plt', skiprows=6, names = colnames)

In [3]:
def add_feat(df_input):
    df = df_input
    # add some initial shifts
    df['lat_shift'] = df.lat.shift(-1)
    df['long_shift'] = df.long.shift(-1)
    df['time_shift'] = df.DateTime.shift(-1)
    # add speed
    def speed(x):
        try:
            s = vincenty((x[-3],x[-2]),(x[0],x[1]), miles = True) / ((x[-1]-x[4]) * 24)
        except:
            s= np.nan
        return s
    df['speed_mph'] = df.apply(speed,axis =1)
    df['speed_shift'] = df.speed_mph.shift(-1)
    # add acceleration
    def accel(x):
        try:
            a = (x[-1] - x[-2]) / ((x[9] - x[4]) *24)
        except:
            a = np.nan
        return a
    df['acceleration'] = df.apply(accel, axis =1)
    df['acceleration_shift'] = df.acceleration.shift(-1)
    # add jerk
    def jerk(x):
        try:
            j = (x[-1] - x[-2]) / ((x[9] - x[4]) *24)
        except:
            j = np.nan
        return a
    df['jerk'] = df.apply(accel, axis =1)
    df['jerk_shift'] = df.jerk.shift(-1)

    def y(x):
        try:
            yy = np.sin((x[8] - x[0]) * np.pi/180) *np.cos( x[7]* np.pi/180)
        except:
            yy= np.nan
        return yy
    df['y'] = df.apply(y, axis =1)

    def x(x):
        try:
            xx = np.cos(x[0] * np.pi/180) *np.sin(x[7]* np.pi/180) - np.sin(x[0]* np.pi/180) * np.cos(x[7]* np.pi/180)*np.cos((x[8]-x[1])* np.pi/180)
        except:
            xx = np.nan
        return xx
    df['x'] = df.apply(x,axis =1)

    def bearing(x):
        try:
            b = np.arctan2(x[-2],x[-1])*180/np.pi
        except:
            b = np.nan
        return b
    df['bearing'] = df.apply(bearing,axis=1)

    df['brearing_shift'] = df.bearing.shift(-1)

    def bearing_rate(x):
        try:
            br = abs(x[-1]-x[-2])
        except:
            br = np.nan
        return br
    df['bearing_rate'] = df.apply(bearing_rate,axis=1)
    return df

In [4]:
# df = add_feat(df)

# df.head()

# for i,name in enumerate(df.quantile(.40).index):
#     print(i , name)

# df.quantile(.40)

In [5]:
def list_df_summary(input_df):
    names = ['speed_1','speed_25','speed_50','speed_75','speed_100','speed_ave',
            'accel_1','accel_25','accel_50','accel_75','accel_100','accel_ave',
            'jerk_1','jerk_25','jerk_50','jerk_75','jerk_100','jerk_ave',
            'bearingRate_1','bearingRate_25','bearingRate_50','bearingRate_75','bearingRate_100','bearingRate_ave']
    values = list()
    #speed
    values.append(input_df.quantile(.01)[8])
    values.append(input_df.quantile(.25)[8])
    values.append(input_df.quantile(.50)[8])
    values.append(input_df.quantile(.75)[8])
    values.append(input_df.quantile(1.0)[8])
    values.append(input_df.mean()[8])
    #accel
    values.append(input_df.quantile(.01)[10])
    values.append(input_df.quantile(.25)[10])
    values.append(input_df.quantile(.50)[10])
    values.append(input_df.quantile(.75)[10])
    values.append(input_df.quantile(1.0)[10])
    values.append(input_df.mean()[10])
    #jerk
    values.append(input_df.quantile(.01)[12])
    values.append(input_df.quantile(.25)[12])
    values.append(input_df.quantile(.50)[12])
    values.append(input_df.quantile(.75)[12])
    values.append(input_df.quantile(1.0)[12])
    values.append(input_df.mean()[12])
    #bearing
    values.append(input_df.quantile(.01)[18])
    values.append(input_df.quantile(.25)[18])
    values.append(input_df.quantile(.50)[18])
    values.append(input_df.quantile(.75)[18])
    values.append(input_df.quantile(1.0)[18])
    values.append(input_df.mean()[18])
    
    return pd.DataFrame([values],columns=names)

In [6]:
plts = []
for folder in glob('/Users/williamyee/Documents/Data Bootcamp/unsupervised-learning-metis/Geolife Trajectories 1.3/Data/*/Trajectory'):
    for file in glob(folder +'/*.plt'):
        plts.append(file)

In [ ]:
large_df = []
for i, file in enumerate(plts):
    print( i / len(plts))
    large_df.append(list_df_summary(add_feat(pd.read_csv(file, skiprows=6,names=colnames))))
    
modeling_df = pd.concat(large_df)

0.0
5.356186395286556e-05
0.00010712372790573112
0.00016068559185859668
0.00021424745581146223
0.0002678093197643278
0.00032137118371719335
0.0003749330476700589
0.00042849491162292447
0.00048205677557579
0.0005356186395286556
0.0005891805034815211
0.0006427423674343867
0.0006963042313872523
0.0007498660953401178
0.0008034279592929834
0.0008569898232458489
0.0009105516871987145
0.00096411355115158
0.0010176754151044456
0.0010712372790573112
0.0011247991430101767
0.0011783610069630423
0.0012319228709159078
0.0012854847348687734
0.001339046598821639
0.0013926084627745045
0.00144617032672737
0.0014997321906802356
0.0015532940546331012
0.0016068559185859668
0.0016604177825388323
0.0017139796464916979
0.0017675415104445634
0.001821103374397429
0.0018746652383502945
0.00192822710230316
0.0019817889662560257
0.0020353508302088912
0.0020889126941617568
0.0021424745581146223
0.002196036422067488
0.0022495982860203535
0.002303160149973219
0.0023567220139260846
0.00241028387787895
0.0024638457418

0.021049812533476164
0.021103374397429032
0.021156936261381896
0.02121049812533476
0.021264059989287626
0.021317621853240494
0.02137118371719336
0.021424745581146223
0.021478307445099088
0.021531869309051956
0.02158543117300482
0.021638993036957686
0.02169255490091055
0.02174611676486342
0.021799678628816283
0.021853240492769148
0.021906802356722013
0.02196036422067488
0.022013926084627745
0.02206748794858061
0.022121049812533475
0.022174611676486343
0.022228173540439208
0.022281735404392072
0.022335297268344937
0.022388859132297805
0.02244242099625067
0.022495982860203535
0.0225495447241564
0.022603106588109267
0.022656668452062132
0.022710230316014997
0.02276379217996786
0.02281735404392073
0.022870915907873594
0.02292447777182646
0.022978039635779324
0.023031601499732192
0.023085163363685057
0.02313872522763792
0.023192287091590786
0.023245848955543654
0.02329941081949652
0.023352972683449384
0.023406534547402248
0.023460096411355116
0.02351365827530798
0.023567220139260846
0.023620

0.04258168184252812
0.04263524370648099
0.04268880557043385
0.04274236743438672
0.042795929298339586
0.04284949116229245
0.042903053026245315
0.042956614890198176
0.043010176754151044
0.04306373861810391
0.043117300482056774
0.04317086234600964
0.04322442420996251
0.04327798607391537
0.04333154793786824
0.0433851098018211
0.04343867166577397
0.04349223352972684
0.0435457953936797
0.043599357257632566
0.043652919121585435
0.043706480985538296
0.043760042849491164
0.043813604713444025
0.04386716657739689
0.04392072844134976
0.04397429030530262
0.04402785216925549
0.04408141403320836
0.04413497589716122
0.04418853776111409
0.04424209962506695
0.04429566148901982
0.044349223352972686
0.04440278521692555
0.044456347080878415
0.04450990894483128
0.044563470808784145
0.04461703267273701
0.044670594536689874
0.04472415640064274
0.04477771826459561
0.04483128012854847
0.04488484199250134
0.04493840385645421
0.04499196572040707
0.04504552758435994
0.0450990894483128
0.04515265131226567
0.0452062

0.06432779860739153
0.0643813604713444
0.06443492233529727
0.06448848419925013
0.064542046063203
0.06459560792715587
0.06464916979110873
0.0647027316550616
0.06475629351901446
0.06480985538296732
0.0648634172469202
0.06491697911087306
0.06497054097482592
0.06502410283877878
0.06507766470273166
0.06513122656668452
0.06518478843063738
0.06523835029459026
0.06529191215854312
0.06534547402249598
0.06539903588644885
0.06545259775040171
0.06550615961435458
0.06555972147830745
0.06561328334226031
0.06566684520621317
0.06572040707016605
0.06577396893411891
0.06582753079807177
0.06588109266202463
0.0659346545259775
0.06598821638993037
0.06604177825388323
0.0660953401178361
0.06614890198178897
0.06620246384574183
0.0662560257096947
0.06630958757364756
0.06636314943760042
0.0664167113015533
0.06647027316550616
0.06652383502945902
0.0665773968934119
0.06663095875736476
0.06668452062131762
0.06673808248527048
0.06679164434922336
0.06684520621317622
0.06689876807712908
0.06695232994108195
0.06700589

0.08655597214783074
0.0866095340117836
0.08666309587573648
0.08671665773968934
0.0867702196036422
0.08682378146759508
0.08687734333154794
0.0869309051955008
0.08698446705945367
0.08703802892340654
0.0870915907873594
0.08714515265131227
0.08719871451526513
0.087252276379218
0.08730583824317087
0.08735940010712373
0.08741296197107659
0.08746652383502945
0.08752008569898233
0.08757364756293519
0.08762720942688805
0.08768077129084093
0.08773433315479379
0.08778789501874665
0.08784145688269952
0.08789501874665238
0.08794858061060525
0.08800214247455812
0.08805570433851098
0.08810926620246384
0.08816282806641672
0.08821638993036958
0.08826995179432244
0.0883235136582753
0.08837707552222818
0.08843063738618104
0.0884841992501339
0.08853776111408677
0.08859132297803964
0.0886448848419925
0.08869844670594537
0.08875200856989823
0.0888055704338511
0.08885913229780397
0.08891269416175683
0.08896625602570969
0.08901981788966257
0.08907337975361543
0.08912694161756829
0.08918050348152115
0.08923406